In [ ]:
# We will do the same analysis as on the 24th for selection 
# of a parcel for a firestation , but use a function
# so that users can change parameters as they wish

In [2]:
# Please create a function to generate buffers
# around schools based on a user specified distance

# bufferSchools("5 Kilometers")

import arcpy

# Let's specify the workspace
arcpy.env.workspace = "C:/EsriTraining/firestation.gdb"

arcpy.Exists(arcpy.env.workspace)




In [3]:
# For ArcGIS Pro 2.7
# How to enable overwriting files
arcpy.env.overwriteOutput = True

def bufferSchools(distance):
    
    arcpy.Buffer_analysis("School", "School_Buffer", distance)
    
bufferSchools("5 Kilometers")

In [ ]:
# For ArcGIS Pro 2.5
def bufferSchools(distance):
    if arcpy.Exists("School_Buffer"):
        arcpy.Delete_management("School_Buffer")
    arcpy.Buffer_analysis("School", "School_Buffer", distance)
    
bufferSchools("5 Kilometers")

In [4]:
# Please create a function to generate buffers
# around the layer specified by the user
# and based on user specified distance

# bufferFeatures("Existing_fire_station", "3 Kilometers")

def bufferFeatures(fc, distance):
    fc_out = fc + "_Buffer"
    arcpy.Buffer_analysis(fc, fc_out, distance)    

bufferFeatures("Existing_fire_station", "3.5 Kilometers")

In [ ]:
# For ArcGIS 2.5

def bufferFeatures(fc, distance):
    fc_out = fc + "_Buffer"
    if arcpy.Exists(fc_out):
        arcpy.Delete_management(fc_out)
    arcpy.Buffer_analysis(fc, fc_out, distance)    

bufferFeatures("Existing_fire_station", "3.5 Kilometers")

In [6]:
# If we have a layer named Existing_fire_station_Buffer,
# how do we learn from which layer it came from

my_string = "Existing_fire_station_Buffer"
original_fc = my_string[:len(my_string)-7]

print(original_fc)

Existing_fire_station


In [7]:
feature_classes = arcpy.ListFeatureClasses()
# print(feature_classes)

# Feature class and layers are interchangable terms in our class
for fc in feature_classes:
    print(fc)

Environmentally_sensitive_area
School
Existing_fire_station
Sites1
Firestation_Buffer
Criteria_Union
Criteria_Dissolve
First_Step
Final_Result
fc1_buffer
fc2_buffer
fc3_buffer
Criteria1_Union
Criteria1_Dissolve
Criteria_1
Criteria_2
Final_Output
Existing_fire_station_Buffer
Environmentally_sensitive_area_Buffer
School_Buffer


In [ ]:
# Create the function for the entire workflow
# For example:
'''
batchFirestationParcelSearch("C:/EsriTraining/firestation.gdb", "Sites1",
                             "Environmentally_sensitive_area", "1.5 Kilometers",
                             "School", "2.3 Kilometers",
                             "Existing_fire_station", "2.5 Kilometers",
                            10000, "Slope", "MEAN < 8", "Final_Output")
'''

In [1]:
def batchFirestationParcelSearch(workspace, input_fc, fc1, b_dist1, fc2, b_dist2, fc3, b_dist3,
                                parcel_size, raster_lyr, raster_criterion, out_fc):
    # Import the main library
    import arcpy
    
    # Let's specify the workspace
    arcpy.env.workspace = workspace
    
    # Make sure you can overwrite layers
    arcpy.env.overwriteOutput = True
    
    # ANALYSES FOR THE FIRST CRITERIA
    
    # Buffer for Environmentally_sensitive_areas, 
    # Schools and Existing Fire Stations    
    arcpy.Buffer_analysis(fc1, fc1 + "_Buffer", b_dist1)
    arcpy.Buffer_analysis(fc2, fc2 + "_Buffer", b_dist2)
    arcpy.Buffer_analysis(fc3, fc3 + "_Buffer", b_dist3)
    
    # Let's do the union analysis
    arcpy.Union_analysis([fc1 + "_Buffer", fc2 + "_Buffer", fc3 + "_Buffer"], 
                         "FirstCriteria_Union")
    
    # Dissolve all union features
    arcpy.Dissolve_management("FirstCriteria_Union", "FirstCriteria_Dissolve")
    
    # Conduct erase analysis
    arcpy.Erase_analysis(input_fc, "FirstCriteria_Dissolve", "First_Step")
    
    # SECOND CRITERIA
    
    # First add area in sq meters
    arcpy.AddField_management("First_Step","area_sq_meters","Double")
    expression1 = "{0}".format("!SHAPE.area@SQUAREMETERS!")       
    arcpy.CalculateField_management("First_Step", "area_sq_meters",
                                    expression1, "PYTHON", )
    
    # Zonal Statistics as Table

    # arcpy.gp.ZonalStatisticsAsTable for 2.5
    # arcpy.sa.ZonalStatisticsAsTable for 2.7
    arcpy.sa.ZonalStatisticsAsTable("First_Step", "OBJECTID", raster_lyr, raster_lyr + "_Statistics",
                                   "NODATA", "MEAN")
    # Join the zonal statistics to the First_Step feature class
    arcpy.JoinField_management("First_Step", "OBJECTID", raster_lyr + "_Statistics", "OBJECTID_1")
    
    # Let's create a feature layer for selection
    arcpy.MakeFeatureLayer_management("First_Step", "Final_Selection_Lyr")
    
    # Define and conduct a query
    query = "area_sq_meters >" + str(parcel_size) + " and OWNER = 'CITY' and " + raster_criterion
    
    arcpy.SelectLayerByAttribute_management("Final_Selection_Lyr", "NEW_SELECTION", query)
    
    arcpy.CopyFeatures_management("Final_Selection_Lyr", out_fc)

In [2]:
batchFirestationParcelSearch("C:/EsriTraining/firestation.gdb", "Sites1",
                             "Environmentally_sensitive_area", "1.4 Kilometers",
                             "School", "2.5 Kilometers",
                             "Existing_fire_station", "2 Kilometers",
                            20000, "Slope", "MEAN < 15", "Final_Output_Lisa")

In [16]:
# for ArcGIS 2.5

def batchFirestationParcelSearchArc2_5(workspace, input_fc, fc1, b_dist1, fc2, b_dist2, fc3, b_dist3,
                                parcel_size, raster_lyr, raster_criterion, out_fc):
    # Import the main library
    import arcpy
    
    # Let's specify the workspace
    arcpy.env.workspace = workspace
    
    # Make sure you can overwrite layers
    # arcpy.env.overwriteOutput = True
    
    # delete all intermediate layers
    intermediate_layers = [fc1 + "_Buffer", fc2 + "_Buffer", fc3 + "_Buffer",
                          "FirstCriteria_Union", "FirstCriteria_Dissolve",
                          "First_Step", raster_lyr + "_Statistics", out_fc]
    for intermediate_layer in intermediate_layers:
        if arcpy.Exists(intermediate_layer):
            arcpy.Delete_management(intermediate_layer)
    
    # ANALYSES FOR THE FIRST CRITERIA
    
    # Buffer for Environmentally_sensitive_areas, 
    # Schools and Existing Fire Stations    
    arcpy.Buffer_analysis(fc1, fc1 + "_Buffer", b_dist1)
    arcpy.Buffer_analysis(fc2, fc2 + "_Buffer", b_dist2)
    arcpy.Buffer_analysis(fc3, fc3 + "_Buffer", b_dist3)
    
    # Let's do the union analysis
    arcpy.Union_analysis([fc1 + "_Buffer", fc2 + "_Buffer", fc3 + "_Buffer"], 
                         "FirstCriteria_Union")
    
    # Dissolve all union features
    arcpy.Dissolve_management("FirstCriteria_Union", "FirstCriteria_Dissolve")
    
    # Conduct erase analysis
    arcpy.Erase_analysis(input_fc, "FirstCriteria_Dissolve", "First_Step")
    
    # SECOND CRITERIA
    
    # First add area in sq meters
    arcpy.AddField_management("First_Step","area_sq_meters","Double")
    expression1 = "{0}".format("!SHAPE.area@SQUAREMETERS!")       
    arcpy.CalculateField_management("First_Step", "area_sq_meters",
                                    expression1, "PYTHON", )
    
    # Zonal Statistics as Table

    # arcpy.gp.ZonalStatisticsAsTable for 2.5
    # arcpy.sa.ZonalStatisticsAsTable for 2.7
    arcpy.sa.ZonalStatisticsAsTable("First_Step", "OBJECTID", raster_lyr, raster_lyr + "_Statistics",
                                   "NODATA", "MEAN")
    # Join the zonal statistics to the First_Step feature class
    arcpy.JoinField_management("First_Step", "OBJECTID", raster_lyr + "_Statistics", "OBJECTID_1")
    
    # Let's create a feature layer for selection
    arcpy.MakeFeatureLayer_management("First_Step", "Final_Selection_Lyr")
    
    # Define and conduct a query
    query = "area_sq_meters >" + str(parcel_size) + " and OWNER = 'CITY' and " + raster_criterion
    
    arcpy.SelectLayerByAttribute_management("Final_Selection_Lyr", "NEW_SELECTION", query)
    
    arcpy.CopyFeatures_management("Final_Selection_Lyr", out_fc)

In [17]:
batchFirestationParcelSearchArc2_5("C:/EsriTraining/firestation.gdb", "Sites1",
                             "Environmentally_sensitive_area", "1.4 Kilometers",
                             "School", "2.5 Kilometers",
                             "Existing_fire_station", "2 Kilometers",
                            20000, "Slope", "MEAN < 15", "Final_Output_Paul_Lisa")